In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/MyDrive/Deepfake_detector/mini_df_data/train_dataset
# %ls
# %cd gdrive
# %ls


/content/gdrive/MyDrive/Deepfake_detector/mini_df_data/train_dataset


In [3]:
#To get the average frame count
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('fake/*.mp4')

# change to path of real videos
# video_files =  glob.glob('real/*.mp4')

#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [300, 301, 300, 300, 301, 300, 300, 300, 300, 300, 300, 300, 301, 300, 301, 300, 300, 300, 301, 300, 300, 300, 301, 300, 167, 300, 300, 300, 300, 300, 300, 300, 301, 301, 300, 300, 301, 300, 300, 300, 300, 301, 301, 167]
Total number of videos:  44
Average frame per video: 294.20454545454544


In [4]:
# # to extract frame
# def frame_extract(path):
#   vidObj = cv2.VideoCapture(path)
#   success = 1
#   while success:
#       success, image = vidObj.read()
#       if success:
#           yield image
!pip3 install face_recognition
# # !mkdir 'content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset/fake'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
import gc
from tqdm import tqdm





def save_cropped_face_frames(path_list, out_dir, max_frames=50):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    already_present_count = len(glob.glob(os.path.join(out_dir, '*.jpg')))
    print("Number of frames already present:", already_present_count)

    for path in tqdm(path_list):
        vidObj = cv2.VideoCapture(path)
        total_frames = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_step = max(1, total_frames // max_frames)  # Calculate step to ensure we get up to max_frames
        frames_saved = 0  # Initialize a counter for saved frames

        try:
            frame_index = 0
            success, frame = vidObj.read()
            while success and frame_index < total_frames:
                if (frame_index % frame_step == 0):  # Only process frame if it's on the step
                    faces = face_recognition.batch_face_locations([frame], number_of_times_to_upsample=0, batch_size=1)
                    for face_locations in faces:
                        for top, right, bottom, left in face_locations:
                            cropped_frame = cv2.resize(frame[top:bottom, left:right], (112, 112))
                            frame_file_path = os.path.join(out_dir, f"{os.path.basename(path)}_frame{frame_index}.jpg")
                            cv2.imwrite(frame_file_path, cropped_frame)
                            frames_saved += 1  # Increment for each saved frame
                success, frame = vidObj.read()
                frame_index += 1
        except Exception as e:
            print(f"Error processing video {path}: {str(e)}")

        vidObj.release()
        cv2.destroyAllWindows()
        gc.collect()  # Explicitly call garbage collector after processing each video
        print(f"Total frames saved for video {os.path.basename(path)}: {frames_saved}")  # Print number of frames saved for each video







In [ ]:
save_cropped_face_frames(video_files, '/content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset_frame/fake')
# save_cropped_face_frames(video_files, '/content/gdrive/MyDrive/Deepfake_detector/dataset/processed_dataset_frame/real')

Number of frames already present: 2206


  2%|▏         | 1/45 [00:30<22:17, 30.40s/it]

Total frames saved for video 4957.mp4: 50


  4%|▍         | 2/45 [00:33<10:27, 14.60s/it]

Total frames saved for video 4955.mp4: 10


...

Total frames saved for video 4999.mp4: 50


 76%|███████▌  | 34/45 [05:16<01:24,  7.68s/it]

Total frames saved for video 4989.mp4: 51


 78%|███████▊  | 35/45 [05:20<01:06,  6.67s/it]

Total frames saved for video 5000.mp4: 51
